In [1]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 3.8MB 50.2MB/s 
     |████████████████████████████████| 512kB 61.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=1b27fae322fc34a945b49500c4169c95886de28571d3c833a5cdd79df7ae835b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installatio

In [18]:
from google.colab import files
files.upload()
# Copy file to the right directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Change permissions owner can read and write
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [19]:
# download the dataset
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 68.1MB/s]
 76% 12.0M/15.9M [00:00<00:00, 20.3MB/s]
100% 15.9M/15.9M [00:00<00:00, 35.8MB/s]
 88% 57.0M/64.7M [00:01<00:00, 32.4MB/s]
100% 64.7M/64.7M [00:01<00:00, 57.9MB/s]
 99% 275M/276M [00:12<00:00, 37.4MB/s]
100% 276M/276M [00:12<00:00, 22.6MB/s]
 57% 9.00M/15.8M [00:00<00:00, 20.3MB/s]
100% 15.8M/15.8M [00:00<00:00, 32.1MB/s]
 41% 5.00M/12.1M [00:00<00:00, 13.0MB/s]
100% 12.1M/12.1M [00:00<00:00, 27.4MB/s]
 41% 5.00M/12.3M [00:00<00:00, 11.1MB/s]
100% 12.3M/12.3M [00:00<00:00, 25.0MB/s]


In [20]:
!unzip test_public_expanded.csv.zip
!unzip train.csv.zip

Archive:  test_public_expanded.csv.zip
  inflating: test_public_expanded.csv  
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [0]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test_public_expanded.csv")

In [0]:
print("Train shape: ", train_df.shape, "\nTest shape: ", test_df.shape)

In [0]:
train_df.head(3)

In [0]:
train_df.columns

In [0]:
train_df.describe()

In [0]:
train_df.drop(['created_date', 'publication_id', 'parent_id', 'article_id',
               'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
               'identity_annotator_count', 'toxicity_annotator_count'], axis=1, inplace=True)

In [24]:
test_df.head(3)

,id,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,identity_annotator_count,toxicity_annotator_count,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability
0,7000000,Jeff Sessions is another one of Trump's Orwell...,2017-01-26 07:37:38.422417+00,13,NaN,164149,approved,0,0,0,4,0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000001,I actually inspected the infrastructure on Gra...,2016-12-03 20:38:21.204649+00,54,655260.0,154341,approved,0,0,0,2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7000002,No it won't . That's just wishful thinking on ...,2017-05-05 18:07:58.560078+00,21,5219683.0,332005,approved,0,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test_df.drop(['created_date', 'publication_id', 'parent_id', 'article_id',
               'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
               'identity_annotator_count', 'toxicity_annotator_count'], axis=1, inplace=True)

In [0]:
test_df.head(3)

In [26]:
print("Average comment_text length is ", np.average([len(i) for i in train_df.comment_text]))

Average comment_text length is  297.2343266067327


In [0]:
MAX_LEN = 300
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = 'target'
IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [0]:
# !pip install emoji
# import emoji
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—' #+ "".join([i for i in emoji.UNICODE_EMOJI])

In [0]:
x_train = train_df[TEXT_COLUMN].astype(str)
y_train = train_df[TARGET_COLUMN].values

In [0]:
y_aux_train = train_df[AUX_COLUMNS].values

In [0]:
lstm_tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
lstm_tokenizer.fit_on_texts(x_train)

In [0]:
# vectorize a text corpus
x_train = lstm_tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)

In [0]:
for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train_df[column] = np.where(train_df[column] >= 0.5, True, False)

In [37]:
!kaggle datasets download -d iezepov/gensim-embeddings-dataset -f glove.840B.300d.gensim.vectors.npy
!kaggle datasets download -d iezepov/gensim-embeddings-dataset -f glove.840B.300d.gensim

 99% 2.24G/2.25G [00:45<00:00, 33.7MB/s]
100% 2.25G/2.25G [00:45<00:00, 52.8MB/s]
 93% 41.0M/44.1M [00:01<00:00, 20.5MB/s]
100% 44.1M/44.1M [00:01<00:00, 24.1MB/s]


In [38]:
!unzip glove.840B.300d.gensim.vectors.npy.zip
!unzip glove.840B.300d.gensim.zip

Archive:  glove.840B.300d.gensim.vectors.npy.zip
  inflating: glove.840B.300d.gensim.vectors.npy  
Archive:  glove.840B.300d.gensim.zip
  inflating: glove.840B.300d.gensim  


In [0]:
def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
          if candidate in embedding_index:
              embedding_matrix[i] = embedding_index[candidate]
              break
    return embedding_matrix

In [40]:
lstm_tokenizer.word_index

{'the': 1,
 'to': 2,
 'and': 3,
 'of': 4,
 'a': 5,
 'is': 6,
 'in': 7,
 'that': 8,
 'I': 9,
 'for': 10,
 'it': 11,
 'you': 12,
 's': 13,
 'are': 14,
 'not': 15,
 'be': 16,
 't': 17,
 'have': 18,
 'on': 19,
 'with': 20,
 'they': 21,
 'as': 22,
 'this': 23,
 'The': 24,
 'was': 25,
 'or': 26,
 'will': 27,
 'their': 28,
 'all': 29,
 'by': 30,
 'but': 31,
 'he': 32,
 'from': 33,
 'we': 34,
 'who': 35,
 'at': 36,
 'can': 37,
 'about': 38,
 'has': 39,
 'your': 40,
 'would': 41,
 'people': 42,
 'do': 43,
 'what': 44,
 'an': 45,
 'more': 46,
 'his': 47,
 'It': 48,
 'like': 49,
 'so': 50,
 'one': 51,
 'no': 52,
 'if': 53,
 'out': 54,
 'Trump': 55,
 'just': 56,
 'up': 57,
 'there': 58,
 'them': 59,
 'our': 60,
 'don': 61,
 'get': 62,
 'when': 63,
 'than': 64,
 'should': 65,
 'You': 66,
 'been': 67,
 'were': 68,
 'my': 69,
 'time': 70,
 'other': 71,
 'any': 72,
 'If': 73,
 'how': 74,
 'only': 75,
 'think': 76,
 'some': 77,
 'because': 78,
 'And': 79,
 'know': 80,
 'those': 81,
 'had': 82,
 'which'

In [41]:
embedding_layer_init_weights = build_matrix(lstm_tokenizer.word_index, "glove.840B.300d.gensim")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [0]:
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

In [0]:
def build_model(embedding_matrix, num_aux_targets):
    words = Input(shape=(None,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    model = Model(inputs=words, outputs=[result, aux_result])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

In [0]:
x_test = test_df[TEXT_COLUMN].astype(str)
y_test = test_df["toxicity"].values

# vectorize TEST text corpus
x_test = lstm_tokenizer.texts_to_sequences(x_test)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [0]:
model = build_model(embedding_layer_init_weights, y_aux_train.shape[-1])

In [0]:
EPOCHS=2
BATCH_SIZE=512

In [59]:
checkpoint_predictions = []
weights = []

model = build_model(embedding_layer_init_weights, y_aux_train.shape[-1])
for global_epoch in range(EPOCHS):
    model.fit(
        x_train,
        [y_train, y_aux_train],
        batch_size=BATCH_SIZE,
        epochs=1
    )
    checkpoint_predictions.append(model.predict(x_test, batch_size=2048)[0].flatten())
    weights.append(2 ** global_epoch)

Epoch 1/1
1804874/1804874 [==============================] - 901s 499us/step - loss: 0.3476 - dense_31_loss: 0.2405 - dense_32_loss: 0.1072 - dense_31_accuracy: 0.7005 - dense_32_accuracy: 0.8547
Epoch 1/1
1804874/1804874 [==============================] - 905s 501us/step - loss: 0.3342 - dense_31_loss: 0.2318 - dense_32_loss: 0.1025 - dense_31_accuracy: 0.7010 - dense_32_accuracy: 0.8550


In [0]:
MODEL_NAME = 'lstm'
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

In [0]:
test_df[MODEL_NAME] = predictions

In [0]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

In [0]:
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

In [0]:
def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

In [0]:
def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

In [0]:
def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

In [0]:
def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [0]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

In [0]:
def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

In [0]:
def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [0]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
TOXICITY_COLUMN = 'toxicity'
TEXT_COLUMN = 'comment_text'

In [74]:
test_df.columns

Index(['id', 'comment_text', 'toxicity', 'severe_toxicity', 'obscene',
       'sexual_explicit', 'identity_attack', 'insult', 'threat', 'male',
       'female', 'transgender', 'other_gender', 'heterosexual',
       'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation',
       'christian', 'jewish', 'muslim', 'hindu', 'buddhist', 'atheist',
       'other_religion', 'black', 'white', 'asian', 'latino',
       'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'lstm'],
      dtype='object')

In [75]:
test_df.lstm

0        0.066435
1        0.010153
2        0.059065
3        0.048402
4        0.732516
           ...   
97315    0.067478
97316    0.008305
97317    0.054102
97318    0.212715
97319    0.013279
Name: lstm, Length: 97320, dtype: float64

In [0]:
from sklearn import metrics
from sklearn import model_selection

In [0]:
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['toxicity'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

In [0]:
test_df = convert_dataframe_to_bool(test_df)

In [0]:
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)

In [82]:
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

0.9267225173026998